In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import warnings

sys.path.append(str(Path.cwd().parent.parent))
from scrappers import WhoScored

warnings.filterwarnings('ignore')

In [2]:
print("Extrayendo schedule de MLS 2025...")
ws = WhoScored(leagues=['USA-MLS'], seasons=['2025'])
schedule = ws.read_schedule()

print(f"\nTotal partidos en MLS 2025: {len(schedule)}")
print(f"\nStages disponibles:")
print(schedule['stage'].value_counts())

Extrayendo schedule de MLS 2025...


INFO     Retrieving calendar for USA-MLS 2025 (Major League Soccer Playoff)

INFO     [1/3] Retrieving fixtures for USA-MLS 2025 (Major League Soccer Playoff)

INFO     [2/3] Retrieving fixtures for USA-MLS 2025 (Major League Soccer Playoff)

INFO     [3/3] Retrieving fixtures for USA-MLS 2025 (Major League Soccer Playoff)

...

INFO     [6/9] Retrieving fixtures for USA-MLS 2025 (Major League Soccer)

INFO     [7/9] Retrieving fixtures for USA-MLS 2025 (Major League Soccer)

INFO     [8/9] Retrieving fixtures for USA-MLS 2025 (Major League Soccer)

INFO     [9/9] Retrieving fixtures for USA-MLS 2025 (Major League Soccer)


Total partidos en MLS 2025: 540

Stages disponibles:
stage
Major League Soccer            510
Major League Soccer Playoff     30
Name: count, dtype: int64


In [3]:
schedule_reset = schedule.reset_index()

miami_variations = ['Inter Miami', 'Inter Miami CF', 'Miami']
mask = pd.Series([False] * len(schedule_reset))

for variation in miami_variations:
    mask |= schedule_reset['home_team'].str.contains(variation, case=False, na=False, regex=False)
    mask |= schedule_reset['away_team'].str.contains(variation, case=False, na=False, regex=False)

miami_matches = schedule_reset[mask].copy()

print(f"Total partidos del Inter Miami: {len(miami_matches)}")
print(f"\nDistribución por stage:")
print(miami_matches['stage'].value_counts())

Total partidos del Inter Miami: 40

Distribución por stage:
stage
Major League Soccer            34
Major League Soccer Playoff     6
Name: count, dtype: int64


In [5]:
miami_ids = miami_matches[[
    'date', 'home_team', 'away_team', 'game_id', 'stage', 
    'home_score', 'away_score', 'status'
]].copy()

miami_ids['date'] = pd.to_datetime(miami_ids['date']).dt.tz_localize(None)
miami_ids = miami_ids.rename(columns={'game_id': 'whoscored_id'})
miami_ids['league'] = 'USA-MLS'
miami_ids['season'] = '25-26'
miami_ids = miami_ids.sort_values('date').reset_index(drop=True)

print(f"\nPrimeros 10 partidos:")
miami_ids.head(10)


Primeros 10 partidos:


,date,home_team,away_team,whoscored_id,stage,home_score,away_score,status,league,season
0,2025-02-23 00:30:00,Inter Miami CF,New York City FC,1876813,Major League Soccer,2,2,6,USA-MLS,25-26
1,2025-03-03 00:00:00,Houston Dynamo FC,Inter Miami CF,1876890,Major League Soccer,1,4,6,USA-MLS,25-26
2,2025-03-09 20:00:00,Inter Miami CF,Charlotte FC,1876837,Major League Soccer,1,0,6,USA-MLS,25-26
3,2025-03-16 23:00:00,Atlanta United,Inter Miami CF,1876903,Major League Soccer,1,2,6,USA-MLS,25-26
4,2025-03-29 23:30:00,Inter Miami CF,Philadelphia Union,1876849,Major League Soccer,2,1,6,USA-MLS,25-26
5,2025-04-06 23:00:00,Inter Miami CF,Toronto FC,1876913,Major League Soccer,1,1,6,USA-MLS,25-26
6,2025-04-13 20:30:00,Chicago Fire FC,Inter Miami CF,1876930,Major League Soccer,0,0,6,USA-MLS,25-26
7,2025-04-19 20:30:00,Columbus Crew,Inter Miami CF,1876933,Major League Soccer,0,1,6,USA-MLS,25-26
8,2025-04-27 21:00:00,Inter Miami CF,FC Dallas,1876954,Major League Soccer,3,4,6,USA-MLS,25-26
9,2025-05-03 23:30:00,Inter Miami CF,New York Red Bulls,1876966,Major League Soccer,4,1,6,USA-MLS,25-26


In [6]:
print("=" * 60)
print("RESUMEN DE PARTIDOS DEL INTER MIAMI - MLS 2025")
print("=" * 60)
print(f"\nTotal de partidos: {len(miami_ids)}")
print(f"\nPor competición:")
for stage, count in miami_ids['stage'].value_counts().items():
    print(f"  - {stage}: {count} partidos")

print(f"\nRango de fechas:")
print(f"  Primer partido: {miami_ids['date'].min()}")
print(f"  Último partido: {miami_ids['date'].max()}")

print(f"\nPartidos como local: {(miami_ids['home_team'].str.contains('Miami', case=False, na=False)).sum()}")
print(f"Partidos como visitante: {(miami_ids['away_team'].str.contains('Miami', case=False, na=False)).sum()}")
print("=" * 60)

RESUMEN DE PARTIDOS DEL INTER MIAMI - MLS 2025

Total de partidos: 40

Por competición:
  - Major League Soccer: 34 partidos
  - Major League Soccer Playoff: 6 partidos

Rango de fechas:
  Primer partido: 2025-02-23 00:30:00
  Último partido: 2025-12-06 19:30:00

Partidos como local: 21
Partidos como visitante: 19


In [7]:
output_path = Path.cwd().parent / 'inter_miami_match_ids_2025.csv'
miami_ids.to_csv(output_path, index=False)

print(f"\nCSV guardado en: {output_path}")
print(f"Columnas: {', '.join(miami_ids.columns.tolist())}")
print(f"\nArchivo listo para usar con viz/match_data.py")


CSV guardado en: /home/jaime/FD/data/blog/inter_miami_match_ids_2025.csv
Columnas: date, home_team, away_team, whoscored_id, stage, home_score, away_score, status, league, season

Archivo listo para usar con viz/match_data.py


In [8]:
miami_ids

,date,home_team,away_team,whoscored_id,stage,home_score,away_score,status,league,season
0,2025-02-23 00:30:00,Inter Miami CF,New York City FC,1876813,Major League Soccer,2,2,6,USA-MLS,25-26
1,2025-03-03 00:00:00,Houston Dynamo FC,Inter Miami CF,1876890,Major League Soccer,1,4,6,USA-MLS,25-26
2,2025-03-09 20:00:00,Inter Miami CF,Charlotte FC,1876837,Major League Soccer,1,0,6,USA-MLS,25-26
3,2025-03-16 23:00:00,Atlanta United,Inter Miami CF,1876903,Major League Soccer,1,2,6,USA-MLS,25-26
4,2025-03-29 23:30:00,Inter Miami CF,Philadelphia Union,1876849,Major League Soccer,2,1,6,USA-MLS,25-26
5,2025-04-06 23:00:00,Inter Miami CF,Toronto FC,1876913,Major League Soccer,1,1,6,USA-MLS,25-26
6,2025-04-13 20:30:00,Chicago Fire FC,Inter Miami CF,1876930,Major League Soccer,0,0,6,USA-MLS,25-26
7,2025-04-19 20:30:00,Columbus Crew,Inter Miami CF,1876933,Major League Soccer,0,1,6,USA-MLS,25-26
8,2025-04-27 21:00:00,Inter Miami CF,FC Dallas,1876954,Major League Soccer,3,4,6,USA-MLS,25-26
9,2025-05-03 23:30:00,Inter Miami CF,New York Red Bulls,1876966,Major League Soccer,4,1,6,USA-MLS,25-26
